In [1]:
%load_ext autoreload
%autoreload

import os
import sys
import argparse
import pdb
import _pickle as cPickle

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from X_evaluate import compute_freq_tokens, compute_perplexity, compute_topic_specialization, compute_hierarchical_affinity
from X_coherence import compute_coherence

# tools

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('-gpu')
parser.add_argument('-path_data', type=str, default='data/bags/instances.pkl')
parser.add_argument('-batch_size', type=int, default=64)
parser.add_argument('-path_model', type=str, default='model/bags/hntm/bags-tree33-temp10-seed1234/model-385000')
parser.add_argument('-index', type=int, default=-4)

_StoreAction(option_strings=['-index'], dest='index', nargs=None, const=None, default=-4, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
option = parser.parse_args(args='-gpu 8'.split())
option.path_config = option.path_model + '.config'
os.environ["CUDA_VISIBLE_DEVICES"] = option.gpu

## load data

In [4]:
_, _, instances, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(option.path_data,'rb'))
batches = get_batches(instances, option.batch_size)

## restore hntm

In [5]:
print('loading %s...' % option.path_model)
config = cPickle.load(open(option.path_config, 'rb'))
model = HierarchicalNeuralTopicModel(config)

with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, option.path_model)

    freq_tokens = compute_freq_tokens(sess, model, bow_idxs, idx_to_word, verbose=True)
    compute_coherence(freq_tokens.values(), config.dir_corpus, topns=[5, 10], verbose=True)
    compute_perplexity(sess, model, batches, verbose=True)
    compute_topic_specialization(sess, model, instances, verbose=True)
    compute_hierarchical_affinity(sess, model, verbose=True)

    sess.close()

loading model/bags/hntm/bags-tree33-temp10-seed1234/model-385000...
INFO:tensorflow:Restoring parameters from model/bags/hntm/bags-tree33-temp10-seed1234/model-385000
0 product price bought quality love time computer buy recommend made
   1 strap pockets shoulder pocket handle inside side compartment nice front
     11 backpack carry room pack books back work pockets space compartments
     12 pocket power mouse charger room cord small carry perfect usb
     14 ipad inch room carry tablet notebook extra perfect laptops dell
   3 inside zipper soft nice material sleeve snug side foam flap
     32 sleeve protection inch netbook neoprene smell padding protect air snug
   2 cover color bottom keyboard love top apple easy plastic screen
     21 pro mac air computer scratches protect protection perfectly hard retina
Average Topic Coherence = 0.108
Perplexity = 481.2
Topic Specialization: depth 1 = 0.26, depth 2 = 0.57, depth 3 = 0.58, 
Hierarchical Affinity: child = 0.25, non-child = 0.07
